In [ ]:
import json
import pandas as pd
import sqlalchemy
import requests

def get_weather(city_df):
    API_key = 'USE YOUR API KEY HERE'
    weather_dict = {'city_id':[],
                'time':[],
                'temp':[],
                'wind_speed':[]}
    for i, city in enumerate(city_df['city_id']):
        url = f"http://api.openweathermap.org/data/2.5/forecast?lat={city_df.iloc[i]['lat']}&lon={city_df.iloc[i]['long']}&appid={API_key}&units=metric"
        response = requests.get(url)
        weather_json = response.json()
        for time in weather_json['list'][:9]:
            weather_dict['city_id'].append(city)
            weather_dict['time'].append(time['dt_txt'])
            weather_dict['temp'].append(time['main']['temp'])
            weather_dict['wind_speed'].append(time['wind']['speed'])
            
    weather_df = pd.DataFrame(weather_dict)
    return weather_df

def lambda_handler(event, context):
    host = 'database-XXXXXXXXXXXXXXXXX'
    schema = 'gans_aws' # The name of your database (schema) might be different
    user = 'admin'
    password = 'USE YOUR PASSWORD HERE'
    port = 3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}' 
    city_df = pd.read_sql('cities',con=con)
    
    weather_df = get_weather(city_df)
    weather_df.to_sql('weather',con=con,if_exists='append',index=False)
  
    # TODO implement
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }